In [84]:
# import libraries
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
import numpy as np
from datetime import datetime
import matplotlib.dates as mdates
from matplotlib import pyplot

In [85]:
def  Actual_Revenue(actual_prices:list(),predicted_prices:list()):
    predicted_len=(len(predicted_prices) - 1)
    predicted_changes = [predicted_prices[i + 1] - predicted_prices[i] for i in range(predicted_len)]
    expected_revenue=0
    actual_revenue=actual_prices[-1]-actual_prices[0]

    for i in range(len(predicted_prices) - 1):  # Assuming a 7-day test period
        if predicted_changes[i] > 0:  # Buy signal
            expected_revenue += predicted_changes[i]

    return expected_revenue/predicted_len,actual_revenue/predicted_len

def  Expected_Revenue(actual_prices:list(),predicted_prices:list()):
    predicted_len=(len(predicted_prices) - 1)
    predicted_changes = [predicted_prices[i + 1] - predicted_prices[i] for i in range(predicted_len)]
    expected_revenue=0
    for i in range(len(predicted_prices)-1):  # Assuming a 7-day test period
        if predicted_changes[i] > 0:  # Buy signal
            expected_revenue += (actual_prices[i + 1] - actual_prices[i])
        elif predicted_changes[i] < 0:  # Sell signal
            expected_revenue += (actual_prices[i] - actual_prices[i + 1])
    return expected_revenue/predicted_len

In [86]:
# load dataset
file_list=['BTC-all.csv','BTC-N2Y.csv','BTC-N4Y.csv']
# file_list=['BTC-all.csv']
# file_list=['ETH-all.csv','ETH-N2Y.csv','ETH-N4Y.csv']

spilt_list=[0.3,0.2,0.1,7]
# spilt_list=[7]


files_list=list()
spilts_list=list()
Classifiers_list=list()

train_mae=list()
train_mape=list()
train_rmse=list()
test_mae=list()
test_mape=list()
test_rmse=list()

Expected_revenues=list()
Actual_revenue=list()
Expected_total_revenues=list()

In [87]:
for file in file_list:
    for splits in spilt_list:
        #dataset
        print(file)
        print(str(splits),end='\n')
        
        path ='data/'+file
        files_list.append(file)
        spilts_list.append(splits)
        Classifiers_list.append('Prophet')

        df = pd.read_csv(path)
        df=df[['Adj Close','Date']]
        df.rename(columns={'Adj Close': 'y','Date':'ds'}, inplace=True)

        if splits==7:
            train_spilt=int(len(df)-splits)
        else:
            train_spilt=len(df)-int(len(df)*splits)
        train_df,test_df=df[:train_spilt],df[train_spilt:]
        

        # m = Prophet(interval_width=0.95, daily_seasonality=True)
        m = Prophet()
        m.fit(train_df)
        future = m.make_future_dataframe(periods=(test_df.shape[0]),freq="D")


        forecast = m.predict(future)

        train_forecast=forecast["yhat"][:train_spilt]
        test_forecast=forecast["yhat"][train_spilt:]
        
        
        Er,Ar=Actual_Revenue(list(test_df['y']),list(test_forecast)) 
        Expected_revenues.append(Er)
        Actual_revenue.append(Ar)
        Expected_total_revenues.append(Expected_Revenue(list(test_df['y']),list(test_forecast)))
        
        
        print('-'*60)
       
        train_mae.append(mean_absolute_error(train_df['y'], train_forecast))
        train_rmse.append(mean_squared_error(train_df['y'], train_forecast)**0.5)
        train_mape.append(mean_absolute_percentage_error(train_df['y'], train_forecast))
        
        test_mae.append(mean_absolute_error(test_df['y'], test_forecast))
        test_rmse.append(mean_squared_error(test_df['y'], test_forecast)**0.5)
        test_mape.append(mean_absolute_percentage_error(test_df['y'], test_forecast))
        print(train_rmse)
        print(test_rmse)
     


BTC-all.csv
0.3


15:46:03 - cmdstanpy - INFO - Chain [1] start processing
15:46:03 - cmdstanpy - INFO - Chain [1] done processing


------------------------------------------------------------
[1456.4174482229585]
[23835.689076684925]
BTC-all.csv
0.2


15:46:04 - cmdstanpy - INFO - Chain [1] start processing
15:46:05 - cmdstanpy - INFO - Chain [1] done processing


------------------------------------------------------------
[1456.4174482229585, 4357.766340356405]
[23835.689076684925, 64671.480227356646]
BTC-all.csv
0.1


...

------------------------------------------------------------
[1456.4174482229585, 4357.766340356405, 4450.04674310948, 5139.648359778404, 4553.914255600728, 4269.729441029571, 4460.37056647803, 5342.42655242778, 4250.47675295296, 2836.0644041462906]
[23835.689076684925, 64671.480227356646, 9044.064280047884, 13733.445221316044, 73760.98104193523, 22753.44198325953, 22115.161820252895, 10665.210711919908, 27833.048907443226, 33093.64742791214]
BTC-N4Y.csv
0.1


15:46:16 - cmdstanpy - INFO - Chain [1] done processing
15:46:16 - cmdstanpy - INFO - Chain [1] start processing


------------------------------------------------------------
[1456.4174482229585, 4357.766340356405, 4450.04674310948, 5139.648359778404, 4553.914255600728, 4269.729441029571, 4460.37056647803, 5342.42655242778, 4250.47675295296, 2836.0644041462906, 3404.6890350666176]
[23835.689076684925, 64671.480227356646, 9044.064280047884, 13733.445221316044, 73760.98104193523, 22753.44198325953, 22115.161820252895, 10665.210711919908, 27833.048907443226, 33093.64742791214, 20014.77367876271]
BTC-N4Y.csv
7


15:46:17 - cmdstanpy - INFO - Chain [1] done processing


------------------------------------------------------------
[1456.4174482229585, 4357.766340356405, 4450.04674310948, 5139.648359778404, 4553.914255600728, 4269.729441029571, 4460.37056647803, 5342.42655242778, 4250.47675295296, 2836.0644041462906, 3404.6890350666176, 2746.576958411601]
[23835.689076684925, 64671.480227356646, 9044.064280047884, 13733.445221316044, 73760.98104193523, 22753.44198325953, 22115.161820252895, 10665.210711919908, 27833.048907443226, 33093.64742791214, 20014.77367876271, 2171.1973684924924]


In [88]:
Summary={'Data':files_list,'Classifier':Classifiers_list,'Test Split':spilts_list,
            'train_mae':train_mae,
            'train_mape':train_mape,
            'train_rmse':train_rmse,          
            'test_mae':test_mae,
            'test_mape':test_mape,
            'test_rmse':test_rmse,
            'Expected_Revenues':Expected_revenues,
            'Actual_Revenue':Actual_revenue,
            'Expected_total_revenues':Expected_total_revenues
         }

df_Summary = pd.DataFrame(Summary)
df_Summary.to_excel("實驗結果/BTC-Prophet Summary NS.xlsx",index=False)  

In [89]:
# fig2 = m.plot_components(forecast)
